In [1]:
from ipynb.fs.full.functions import getPatientData, getCancerPixels, getCombinedCancerMask

import matplotlib.pyplot as plt
import scipy.stats
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
import math

In [3]:
[patients, numPatients, numPatientsWithTumor] = getPatientData()
patients = [item for item in patients if item["patientID"] != "P00000249"]
adc_confusion_matrix_average = np.zeros(shape=(2,2))
cdi_confusion_matrix_average = np.zeros(shape=(2,2))

kf = KFold(n_splits=2, shuffle=True)
for patients_train, patients_test in kf.split():
    adc_threshold, cdi_threshold = train(patients_train)
    adc_confusion_matrix, cdi_confusion_matrix = evaluate(patients_test,adc_threshold, cdi_threshold)
    for i in range(4):
        adc_confusion_matrix_average[i] += adc_confusion_matrix[i]
        cdi_confusion_matrix_average[i] += cdi_confusion_matrix[i]

for i in range(4):
    adc_confusion_matrix_average[i] /= len(adc_confusion_matrix[i]
    cdi_confusion_matrix_average[i] /= len(cdi_confusion_matrix[i]



SyntaxError: unexpected EOF while parsing (<ipython-input-3-8ce8bd4f412d>, line 10)

In [322]:
adcCancerPixels,    cdiCancerPixels    = np.array([]), np.array([])
adcNonCancerPixels, cdiNonCancerPixels = np.array([]), np.array([])

for patient in patients:
    if patient["numTumor"] != 0:
        cancerPixelsTmp = getCancerPixels(patient,"adc",True)
        adcCancerPixels = np.append(adcCancerPixels, cancerPixelsTmp)
        cancerPixelsTmp = getCancerPixels(patient,"cdi",True)
        cdiCancerPixels = np.append(cdiCancerPixels, cancerPixelsTmp)
    nonCancerPixelsTmp = getCancerPixels(patient,"adc",False)
    adcNonCancerPixels = np.append(adcNonCancerPixels, nonCancerPixelsTmp)
    nonCancerPixelsTmp = getCancerPixels(patient,"cdi",False)
    cdiNonCancerPixels = np.append(cdiNonCancerPixels, nonCancerPixelsTmp)

In [323]:
dist = scipy.stats.norm

mu_adc_cancer,     std_adc_cancer     = dist.fit(adcCancerPixels)
mu_adc_non_cancer, std_adc_non_cancer = dist.fit(adcNonCancerPixels)

x_adc_cancer_lower_bound = mu_adc_cancer    -4*std_adc_cancer
x_adc_cancer_upper_bound = mu_adc_cancer    +4*std_adc_cancer
x_adc_non_cancer_lower_bound = mu_adc_non_cancer-4*std_adc_non_cancer
x_adc_non_cancer_upper_bound = mu_adc_non_cancer+4*std_adc_non_cancer

x_adc_lower_bound = x_adc_cancer_lower_bound if x_adc_cancer_lower_bound < x_adc_non_cancer_lower_bound else x_adc_non_cancer_lower_bound
x_adc_upper_bound = x_adc_cancer_upper_bound if x_adc_cancer_upper_bound > x_adc_non_cancer_upper_bound else x_adc_non_cancer_upper_bound

x_adc = np.linspace(x_adc_lower_bound, x_adc_upper_bound, 200)

y_adc_cancer     = dist.pdf(x_adc,    mu_adc_cancer,    std_adc_cancer)
y_adc_non_cancer = dist.pdf(x_adc,mu_adc_non_cancer,std_adc_non_cancer)

plt.figure(figsize=(10,7))
plt.plot(x_adc,    y_adc_cancer)
plt.plot(x_adc,y_adc_non_cancer)
plt.title("ADC")
plt.xlabel("Intensity")
plt.ylabel("P( Intensity )")
plt.legend(["Cancer","Non-cancer"])
plt.show()

In [324]:
dist = scipy.stats.norm

mu_cdi_cancer,     std_cdi_cancer     = dist.fit(np.log(cdiCancerPixels[cdiCancerPixels>0]))
mu_cdi_non_cancer, std_cdi_non_cancer = dist.fit(np.log(cdiNonCancerPixels[cdiNonCancerPixels>0]))

x_cdi_cancer_lower_bound = mu_cdi_cancer    -4*std_cdi_cancer
x_cdi_cancer_upper_bound = mu_cdi_cancer    +4*std_cdi_cancer
x_cdi_non_cancer_lower_bound = mu_cdi_non_cancer - 4*std_cdi_non_cancer
x_cdi_non_cancer_upper_bound = mu_cdi_non_cancer + 4*std_cdi_non_cancer

x_cdi_lower_bound = x_cdi_cancer_lower_bound if x_cdi_cancer_lower_bound < x_cdi_non_cancer_lower_bound else x_cdi_non_cancer_lower_bound
x_cdi_upper_bound = x_cdi_cancer_upper_bound if x_cdi_cancer_upper_bound > x_cdi_non_cancer_upper_bound else x_cdi_non_cancer_upper_bound

x_cdi     = np.linspace(x_cdi_lower_bound, x_cdi_upper_bound, 200)

y_cdi_cancer     = dist.pdf(x_cdi,    mu_cdi_cancer,    std_cdi_cancer)
y_cdi_non_cancer = dist.pdf(x_cdi,mu_cdi_non_cancer,std_cdi_non_cancer)

plt.figure(figsize=(10,7))
plt.plot(x_cdi,    y_cdi_cancer)
plt.plot(x_cdi,y_cdi_non_cancer)
plt.title("CDI")
plt.xlabel("log(Intensity)")
plt.ylabel("P( log(Intensity) )")
plt.legend(["Cancer","Non-cancer"])
plt.show()

In [325]:
adc_threshold = x_adc[np.argmin(np.abs(y_adc_cancer / y_adc_non_cancer -1))]
cdi_threshold = x_cdi[np.argmin(np.abs(y_cdi_cancer / y_cdi_non_cancer -1))]
print("ADC Decision boundary:")
print(f'Cancer: x < {adc_threshold} Non-Cancer: x > {adc_threshold}')
print("CDI Decision boundary:")
print(f'Cancer: x > {cdi_threshold} Non-Cancer: x > {cdi_threshold}')

### Confusion Matrix

In [326]:
patient_not_found = True
for patient in patients_test:
    if patient["patientID"] == "P00000006":
        patient_not_found = False
        cdi_prediction = np.zeros(shape=patient["pMask"].shape, dtype=bool)
        cdi_threshold = math.exp(cdi_threshold)
        cdi_prediction[patient["cdi"] > cdi_threshold] = True
        cdi_prediction[patient["cdi"] < cdi_threshold] = False
        patient["cdiPrediction"] = cdi_prediction
        tn, fp, fn, tp = confusion_matrix(patient["pMask"].flatten(), patient["cdiPrediction"].flatten()).ravel()
        print(f'sensitivity = {tp / (tp + fn):.3f} specificity = {tn / (tn + fp):.3f} accuracy = {(tp + tn) / (tp + tn + fp + fn):.3f}')
        print(f'tn = {tn} fp = {fp} fn = {fn} tp = {tp}')
if patient_not_found:
    print("patient not found")